###  Defining the business goal: 

Construction of a Machine Learning model with >80% accuracy capable of classify whether a user review has positive, negative or neutral connotation, in order to adjust your marketing campaigns


###  Dataset:

Data set used was IMBD (Internet Movie Database) of 50K Movie reviews from Kaggle.
Link: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews )


### Libraries

In [2]:
import re
import pickle
import nltk
import sklearn
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


### Upload dataset and Exploratory Analysis

In [3]:
df = pd.read_csv('IMDBDataset.csv')

In [4]:
df.shape

(50000, 2)

In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [7]:
df.sentiment.value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

### Data cleaning


In [9]:
df.sentiment.replace('positive', 1, inplace = True)
df.sentiment.replace('negative', 0, inplace = True)

In [10]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [11]:
df.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [12]:
# Function to clean text strings and remove HTML tags
def df_clean_data(text):
    cleaned = re.compile(r'<.*?>')
    return re.sub(cleaned, '', text)

In [13]:
# Testing Function
text_with_tags = "<p>This is an example <b>with</b> tags HTML.</p>"
text_clean = df_clean_data(text_with_tags)
print(text_clean)

This is an example with tags HTML.


In [14]:
# Applying function on dataset
df.review = df.review.apply(df_clean_data)
df.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [17]:
# Function to clean special character
# if i.isalnum() verify if character i is numeric
def df_clean_special_character(text):
    rem = ''
    for i in text:
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
            
    return rem

In [18]:
# Testing function
text_with_special_character= "Hello, world! What's up?"
text_clean = df_clean_special_character(text_with_special_character)
print(text_clean)

Hello  world  What s up 


In [19]:
# Applying function
df.review = df.review.apply(df_clean_special_character)
df.review[0]

'One of the other reviewers has mentioned that after watching just 1 Oz episode you ll be hooked  They are right  as this is exactly what happened with me The first thing that struck me about Oz was its brutality and unflinching scenes of violence  which set in right from the word GO  Trust me  this is not a show for the faint hearted or timid  This show pulls no punches with regards to drugs  sex or violence  Its is hardcore  in the classic use of the word It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary  It focuses mainly on Emerald City  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  Em City is home to many  Aryans  Muslims  gangstas  Latinos  Christians  Italians  Irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [20]:
# Function to convert text to lower case in order to standardize data
def df_convert_lower_case(text):
    return text.lower()

In [21]:
df.review = df.review.apply(df_convert_lower_case)
df.review[0]

'one of the other reviewers has mentioned that after watching just 1 oz episode you ll be hooked  they are right  as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence  which set in right from the word go  trust me  this is not a show for the faint hearted or timid  this show pulls no punches with regards to drugs  sex or violence  its is hardcore  in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  em city is home to many  aryans  muslims  gangstas  latinos  christians  italians  irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [22]:
# Function to remove stopwords
# stopwords.words('english') set of stop words in english like common words "the","is", "in", etc 
def df_remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(str(text))
    return [w for w in words if w not in stop_words]

In [23]:
# Testing function
phrase = "They are right, as this is exactly what happened with me."
phrase_output = df_remove_stopwords(phrase)
print(phrase_output)

['They', 'right', ',', 'exactly', 'happened', '.']


In [24]:
# Applying function
df.review = df.review.apply(df_remove_stopwords)
df.review[0]

['one',
 'reviewers',
 'mentioned',
 'watching',
 '1',
 'oz',
 'episode',
 'hooked',
 'right',
 'exactly',
 'happened',
 'first',
 'thing',
 'struck',
 'oz',
 'brutality',
 'unflinching',
 'scenes',
 'violence',
 'set',
 'right',
 'word',
 'go',
 'trust',
 'show',
 'faint',
 'hearted',
 'timid',
 'show',
 'pulls',
 'punches',
 'regards',
 'drugs',
 'sex',
 'violence',
 'hardcore',
 'classic',
 'use',
 'word',
 'called',
 'oz',
 'nickname',
 'given',
 'oswald',
 'maximum',
 'security',
 'state',
 'penitentary',
 'focuses',
 'mainly',
 'emerald',
 'city',
 'experimental',
 'section',
 'prison',
 'cells',
 'glass',
 'fronts',
 'face',
 'inwards',
 'privacy',
 'high',
 'agenda',
 'em',
 'city',
 'home',
 'many',
 'aryans',
 'muslims',
 'gangstas',
 'latinos',
 'christians',
 'italians',
 'irish',
 'scuffles',
 'death',
 'stares',
 'dodgy',
 'dealings',
 'shady',
 'agreements',
 'never',
 'far',
 'away',
 'would',
 'say',
 'main',
 'appeal',
 'show',
 'due',
 'fact',
 'goes',
 'shows',
 'da

In [25]:
# Function stemmer
# "Stemming" is a Neural Language Processing (NLP) thecnique used to reduce words 
# to their root or base form by removing common prefixes and suffixes.

def df_stemmer(text):
    object_stemmer = SnowballStemmer('english')
    return " ".join([object_stemmer.stem(w) for w in text])

In [26]:
# Testing function
text = "The cats are running"
text_stemmed = df_stemmer(text)
print(text_stemmed)

t h e   c a t s   a r e   r u n n i n g


In [27]:
# Applying function
df.review = df.review.apply(df_stemmer)
df.review[0]

'one review mention watch 1 oz episod hook right exact happen first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word call oz nicknam given oswald maximum secur state penitentari focus main emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch darker side'

### Data Pre-Processing

In [28]:
pd.set_option('display.max_colwidth', 120)

In [29]:
# Extract review text (input)
x = np.array(df.iloc[:,0].values)

In [30]:
# Extract review feeling (output)
y = np.array(df.sentiment.values)

In [31]:
# Divide dataset in training and teste 80/20
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2 , random_state = 0)

In [32]:
type(x_train)

numpy.ndarray

In [33]:
# creating a vector (convert text data in numerical representation )
vectorizer_df = CountVectorizer(max_features = 1000)

In [34]:
# Fit and transform vectorizer with training data
x_train_final = vectorizer_df.fit_transform(x_train).toarray()

In [35]:
# Just transform in data test
x_test_final = vectorizer_df.fit_transform(x_test).toarray()

In [36]:
print("x_train_final:", x_train_final.shape)
print("x_test_final:", x_test_final.shape)

x_train_final: (40000, 1000)
x_test_final: (10000, 1000)


In [37]:
print(x_train_final)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [38]:
print("x_test_final:", x_test_final.shape)
print("y_test:", y_test.shape)

x_test_final: (10000, 1000)
y_test: (10000,)


### Machine Learning Modeling

#### Gaussian Naive Bayes

In [39]:
model_v1 = GaussianNB()
model_v1.fit(x_train_final, y_train)

GaussianNB()

#### Multinomial Naive Bayes

In [40]:
model_v2 = MultinomialNB(alpha = 1.0, fit_prior = True)
model_v2.fit(x_train_final, y_train)

MultinomialNB()

#### Bernoulli Naive Bayes

In [41]:
model_v3 = BernoulliNB(alpha = 1.0, fit_prior = True)
model_v3.fit(x_train_final, y_train)

BernoulliNB()

### Evaluation and comparing ML models

In [42]:
# Predict with data test
ypred_v1 = model_v1.predict(x_test_final)
ypred_v2 = model_v2.predict(x_test_final)
ypred_v3 = model_v3.predict(x_test_final)

In [43]:
# Checking accuracy by accuracy_score:
print("Accuracy of GaussianNB model = ", accuracy_score(y_test, ypred_v1) * 100)
print("Accuracy of MultinomialNB model = ", accuracy_score(y_test, ypred_v2) * 100)
print("Accuracy of BernoulliNB model = ", accuracy_score(y_test, ypred_v3) * 100)


Accuracy of GaussianNB model =  64.03999999999999
Accuracy of MultinomialNB model =  73.32
Accuracy of BernoulliNB model =  74.29


In [49]:
# Checking accuracy by AUC (Area Under The Curve)
# AUC for GaussianNB
y_prob = model_v1.predict_proba(x_test_final)[:,1]
auc = roc_auc_score(y_test, y_prob)
print("AUC GaussianNB =", auc)

AUC GaussianNB = 0.6668474355243407


In [50]:
# AUC for MultinomialNB
y_prob = model_v2.predict_proba(x_test_final)[:,1]
auc = roc_auc_score(y_test, y_prob)
print("AUC MultinomialNB =", auc)

AUC MultinomialNB = 0.8071783917411954


In [51]:
# AUC for BernoulliNB
y_prob = model_v3.predict_proba(x_test_final)[:,1]
auc = roc_auc_score(y_test, y_prob)
print("AUC BernoulliNB =", auc)

AUC BernoulliNB = 0.8188667244694989


In [52]:
# Save best ML model at disk:
with open('model_v3_bernoulliNB.pkl', 'wb') as archive:
    pickle.dump(model_v3, archive)